<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/r_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# why use R here?  
# edgarWebR pulls sections really well
# do i want to find a python library for the same thing?  sure.
# do i want to write a python utils file for the same thing?  not really but we may have to!

#first pull the ticker list with google drive handler lib
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
from colab_util import *
drive_handler = GoogleDriveHandler()

#drive_handler.download('test_ticker_list.csv', target_path='test_ticker_list.csv')
drive_handler.download('implementation_ticker_list.csv', target_path='implementation_ticker_list.csv')

#import pandas as pd
#df_tickers = pd.read_csv('test_ticker_list.csv')
#df_tickers = df_tickers[df_tickers['Symbol']=='FB']
#df_tickers.to_csv('test_ticker_list.csv')


Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [3]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [12]:
%%R

#R GETTER.  For this example, only one ticker (head,1) is pulled 
#file is saved into local 
#then python code cell pushes file to google sheet
#devtools::install_github("tidyverse/googlesheets4")

#devtools::install_version("xml2", version = "1.2.2", repos = "http://cran.us.r-project.org")
#file creates a set of csv from ticker list which include metadata & text data.


  #devtools::install_packages('trinker/textclean')
  devtools::install_github("mwaldstein/edgarWebR")
  devtools::install_github("r-lib/xml2") #this for edgarWebR 
  devtools::install_github('trinker/textclean')

  library(edgarWebR) #this is an up to date library with an active maintainer.
  library(xml2)
  library(knitr)
  library(dplyr)
  library(purrr)
  library(rvest)
  library(tidyr)
  library(readr)
  #library(textshape)
  #library(lexicon)
  library(textclean)


  LOGFILE = format(Sys.time(), "%b_%d_%Y.log")
  print(LOGFILE)

  CSVFILE = format(Sys.time(), "%b_%d_%Y.csv")
  print(CSVFILE)

  get_filings_links <-function(str_ticker) {
    df_filings <- company_filings(str_ticker, type = "10-", count = 20)
    df_filings <- df_filings[df_filings$type == "10-K" | df_filings$type == "10-Q", ]
    df_filing_infos <- map_df(df_filings$href, filing_information)
    df_filings <- bind_cols(df_filings, df_filing_infos)
    return(head(as_tibble(df_filings),12))
  }

  write_log <- function(str_text) {
      print(str_text)
      if (file.exists(LOGFILE)) {
          write(str_text,file=LOGFILE,append=TRUE)
      } else {
          write(str_text,file=LOGFILE,append=FALSE)
      }

  }

  write_log_csv <- function(df) {
      if (file.exists(CSVFILE)) {
          write_csv(df,CSVFILE,append=TRUE)
      } else {
          write_csv(df,file=CSVFILE,append=FALSE)
      }

  }

R[write to console]: Skipping install of 'edgarWebR' from a github remote, the SHA1 (e7fa70ea) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'xml2' from a github remote, the SHA1 (876759f3) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'textclean' from a github remote, the SHA1 (184d7868) has not changed since last install.
  Use `force = TRUE` to force installation



[1] "May_18_2020.log"
[1] "May_18_2020.csv"


In [13]:
%%R

get_mdna_text <- function(str_href) {
  write_log(str_href)
  
  #default search
  str_section = 'item 2|item 7'
  str_search = 'discussion'  
  
  df_filing_documents <- filing_documents(str_href) %>%
    filter(!grepl('.pdf',href))

  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)

  #return(doc)
    
  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #if default search fails, use a dictionary attempt
  if (nrow(df_txt) == 0) {
    write_log('going to backup')
    # paired vector of start and ending text to slice if found

    #JPM and #ECL
    vec_start_end <- c('EXECUTIVE OVERVIEW'='CONSOLIDATED RESULTS OF OPERATIONS',
                       'The following management discussion and analysis'='NON-GAAP FINANCIAL MEASURES',
                       'CURRENT ECONOMIC CONDITIONS'='FORWARD-LOOKING STATEMENTS')
    
    #this would be case sensitive
    for (start_text in names(vec_start_end)) {
      
      end_text = as.character(vec_start_end[start_text])
      
      write_log(paste0('trying ',start_text))
      write_log(paste0('to ',end_text))
      
      i_start = as.integer(which(grepl(start_text, doc$text))) #this should be a loop for each dictionary item
      i_end = as.integer(which(grepl(end_text, doc$text)))
      
      
      write_log(i_start)
      write_log(i_end)
      
      #return(i_start)
      
      if (length(i_start) != 0 & length(i_end) != 0) {
        df_txt = doc[i_start:i_end,]
      }
      
    }
    
  }
  #we could do some text preprocessing here.
  
  df_txt <- as_tibble(df_txt) %>%
    #mutate(text = textclean::strip(text)) %>%
    mutate(section = str_search)
  
  return(df_txt)
}

get_section_text <- function(str_href, str_section, str_search) {
  write_log(str_href)
  
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)
  
  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #we could do some text preprocessing here.
  
  df_txt <- as_tibble(df_txt) %>%
    mutate(text = textclean::strip(text)) %>%
    mutate(section = str_search)
  
  return(df_txt)
}


get_document_text <- function(str_ticker, force = FALSE) { #not using force yet
  start_time <- Sys.time()
  
  write_log(str_ticker)
  
  str_write_name <- paste0('sec_data_folder/',str_ticker)
  
  write_log("get filings links ...")
  
  df_filings <- get_filings_links(str_ticker) %>% 
        mutate(ticker = str_ticker)
  
  write_log_csv(df_filings)
#for debug
  #return(NULL)
  
  if (length(df_filings == 0)) {
      return(NULL)
  }

  write_log("get section text ...")
  
  df_data <- (df_filings) %>% 
    rowwise() %>%
    mutate(nest_discussion = map(.x = href, .f = get_mdna_text)) %>%
    ungroup() %>%
    group_by(period_date) %>%
    arrange(desc(period_date))
  
  #jenky - find a rowwise application
  a <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_discussion) %>%
    unnest(nest_discussion)

  write_log("write to local csv  ...")
  df_data <- a %>%
    as_tibble() %>%
    write_csv(paste0(str_write_name,".csv"))
  
  end_time <- Sys.time()
  
  write_log(end_time - start_time)
  
  return(df_data)
}

#long run.
df_tickers <- read_csv('implementation_ticker_list.csv')
dir.create('sec_data_folder', showWarnings = FALSE)

#file creates a set of csv from ticker list which include metadata & text data.
df_tickers <- (df_tickers)

df_data <- map_df(df_tickers$Symbol, get_document_text)
#print(head(df_data))

R[write to console]: Parsed with column specification:
cols(
  X1 = col_double(),
  Symbol = col_character(),
  Security = col_character(),
  `SEC filings` = col_character(),
  `GICS Sector` = col_character(),
  `GICS Sub Industry` = col_character(),
  `Headquarters Location` = col_character(),
  `Date first added` = col_character(),
  CIK = col_double(),
  Founded = col_character(),
  mkt_cap = col_double()
)



[1] "MMM"
[1] "get filings links ..."
[1] "ABT"
[1] "get filings links ..."
[1] "ABBV"
[1] "get filings links ..."
[1] "ABMD"
[1] "get filings links ..."
[1] "ACN"
[1] "get filings links ..."
[1] "ATVI"
[1] "get filings links ..."
[1] "ADBE"
[1] "get filings links ..."
[1] "AMD"
[1] "get filings links ..."
[1] "AAP"
[1] "get filings links ..."
[1] "AES"
[1] "get filings links ..."
[1] "AFL"
[1] "get filings links ..."
[1] "A"
[1] "get filings links ..."
[1] "APD"
[1] "get filings links ..."
[1] "AKAM"
[1] "get filings links ..."
[1] "ALK"
[1] "get filings links ..."
[1] "ALB"
[1] "get filings links ..."
[1] "ARE"
[1] "get filings links ..."
[1] "ALXN"
[1] "get filings links ..."
[1] "ALGN"
[1] "get filings links ..."
[1] "ALLE"
[1] "get filings links ..."
[1] "AGN"
[1] "get filings links ..."
[1] "ADS"
[1] "get filings links ..."
[1] "LNT"
[1] "get filings links ..."
[1] "ALL"
[1] "get filings links ..."
[1] "GOOG"
[1] "get filings links ..."
[1] "MO"
[1] "get filings links ..."
[1] "A

In [5]:
#push files to google drive
import os.path
from os import path
from colab_util import *
drive_handler = GoogleDriveHandler()

sec_folder_id = drive_handler.create_folder('sec_data_folder')

import pandas as pd
df_tickers = pd.read_csv('implementation_ticker_list.csv')

for i, row in df_tickers.iterrows():
  str_ticker = row['Symbol']
  
  print('working for:'+str_ticker+".csv ...")

  str_to_file = 'sec_data_folder'
  str_from_file = 'sec_data_folder/'+str_ticker+'.csv'

  if path.exists(str_from_file): # a rare example of error handling
    drive_handler.upload(str_from_file, parent_path=str_to_file,overwrite = True)

print("done!")


sec_data_folder already exists
working for:MMM.csv ...


AttributeError: ignored